In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

from datasets.img_dataset import FoodImgs
from models.cnn import SnapSnack

In [7]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
import math
from torchvision import transforms as T

transforms = T.Compose(
                [
                    T.Resize(256),
                    T.CenterCrop(224),
                    T.ToTensor(),
                    T.Normalize(
                        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                    ),
                ]
            )

dataset = FoodImgs(
    transforms = transforms,
)

s = len(dataset)
train_size = int(math.ceil(s * 0.7))
test_size = int(math.ceil(s * 0.15))
val_size = s - test_size - train_size

train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])
print(f"{len(train_set)} train imgs \n{len(val_set)} val imgs\n{len(test_set)} test imgs")

6363 train imgs 
1363 val imgs
1364 test imgs


In [ ]:
from pytorch_lightning.loggers import WandbLogger
import wandb
# wandb.init(project="snapnsnack")
wandb_logger = WandbLogger(project='snapnsnack')
model = SnapSnack(fc_layers=(), output_dim=4, lr=1)
trainer = pl.Trainer(logger=wandb_logger, max_epochs=5)
trainer.fit(model, train_dataloader=DataLoader(train_set, batch_size=3), val_dataloaders=DataLoader(val_set), )

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


avg_val_loss,170.90648
_step,2123
_runtime,1436
_timestamp,1607120242
loss,0.00963
batch_nb,2120
r2_calories,-23851.26439
r2_proteins,-755.72677
r2_fat,-80006.12588
r2_sodium,-6839.26059
r2_overall,-27863.09441


avg_val_loss,▁█
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch_nb,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
r2_calories,▁███████████████████████████████████████
r2_proteins,▆▆████▁██████████████████████████▇██████
r2_fat,▁▆▆▇█████████████████▂█████████████████▆
r2_sodium,▁▂█▃██▆█████████████████████████████████
r2_overall,▁▆▇▇██▇██████████████▇██████████████████


wandb: wandb version 0.10.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade



  | Name     | Type   | Params
------------------------------------
0 | backbone | ResNet | 11 M  
1 | softmax  | Tanh   | 0     
/opt/anaconda3/envs/py36/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/opt/anaconda3/envs/py36/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The validation_epoch_end should not return anything as of 9.1.to log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)
/opt/anaconda3/envs/py36/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


wandb: WARNING Step must only increase in log calls.  Step 49 < 51; dropping {'train_loss': 105.55664825439453, 'epoch': 0}.
wandb: WARNING Step must only increase in log calls.  Step 99 < 101; dropping {'train_loss': 5.618946552276611, 'epoch': 0}.
wandb: WARNING Step must only increase in log calls.  Step 149 < 151; dropping {'train_loss': 8.415776252746582, 'epoch': 0}.
wandb: WARNING Step must only increase in log calls.  Step 199 < 201; dropping {'train_loss': 0.2595556080341339, 'epoch': 0}.
wandb: WARNING Step must only increase in log calls.  Step 249 < 251; dropping {'train_loss': 0.5947554707527161, 'epoch': 0}.
wandb: WARNING Step must only increase in log calls.  Step 299 < 301; dropping {'train_loss': 0.6587228178977966, 'epoch': 0}.
wandb: WARNING Step must only increase in log calls.  Step 349 < 351; dropping {'train_loss': 0.11000121384859085, 'epoch': 0}.
wandb: WARNING Step must only increase in log calls.  Step 399 < 401; dropping {'train_loss': 0.4919241964817047, '

wandb: WARNING Step must only increase in log calls.  Step 2149 < 2153; dropping {'train_loss': 10.158279418945312, 'epoch': 1}.
wandb: WARNING Step must only increase in log calls.  Step 2199 < 2203; dropping {'train_loss': 0.6045412421226501, 'epoch': 1}.


In [25]:
import torch
import pickle as pkl
import pandas as pd
epi = pd.read_csv("../data/snapnsnackdb/epi_r.csv")

epi.head()

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.0,6.0,7.0,165.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,547.0,20.0,32.0,452.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
img_4 = "/Users/davinan/Dropbox/github/Snap-N-Snack/data/snapnsnackdb/simple_images/Spinach+Noodle+Casserole+/Spinach+Noodle+Casserole+_1.jpeg"

In [35]:
import matplotlib.pyplot as plt
im = plt.imread(img_4)

In [43]:
from torchvision import transforms as T
from PIL import Image

transform = T.Compose(
            [
                T.ToTensor(),
                T.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
                T.Resize(256),
                T.CenterCrop(224),
            ]
        )

model.eval()
oi = transform(Image.fromarray(im)).unsqueeze(0)
preds = model(oi)

In [44]:
preds

tensor([[  3.7612, 327.7388,  11.6335,  22.7230, 274.9613]],
       grad_fn=<AddmmBackward>)

In [5]:
import torch
torch.__version__

'1.7.0'

In [10]:
import torchvision
torchvision.__version__

'0.8.1'

In [35]:
epi = epi.dropna()
epi

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.0,6.0,7.0,165.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,547.0,20.0,32.0,452.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,The Best Blts,4.375,948.0,19.0,79.0,1042.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,Parmesan Puffs,3.125,28.0,2.0,2.0,64.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20048,Artichoke and Parmesan Risotto,4.375,671.0,22.0,28.0,583.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20049,Turkey Cream Puff Pie,4.375,563.0,31.0,38.0,652.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
20050,Snapper on Angel Hair with Citrus Cream,4.375,631.0,45.0,24.0,517.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
epi.std()

rating             1.285518
calories      359848.417868
protein         3843.462312
fat            20459.329549
sodium        334042.078448
                  ...      
cookbooks          0.011228
leftovers          0.017751
snack              0.036360
snack week         0.028615
turkey             0.144198
Length: 679, dtype: float64

In [28]:
epi.mean()

rating           3.760952
calories      6350.682993
protein        100.324571
fat            346.986826
sodium        6252.742310
                 ...     
cookbooks        0.000126
leftovers        0.000315
snack            0.001324
snack week       0.000819
turkey           0.021243
Length: 679, dtype: float64

In [48]:
train_set[0]

(tensor([[[ 1.4612,  1.4620,  1.4876,  ...,  1.1964,  1.2565,  1.2898],
          [ 1.4604,  1.4612,  1.4630,  ...,  1.2236,  1.2581,  1.2907],
          [ 1.4466,  1.4481,  1.4550,  ...,  1.2609,  1.2956,  1.3032],
          ...,
          [ 1.5514,  1.3198,  1.4637,  ...,  1.7522,  1.7191,  1.7283],
          [ 1.4293,  1.4341,  1.5262,  ...,  1.7638,  1.7646,  1.7371],
          [ 1.3825,  1.4438,  1.5395,  ...,  1.7559,  1.7344,  1.6855]],
 
         [[ 1.6758,  1.6766,  1.7028,  ...,  1.4082,  1.4665,  1.5006],
          [ 1.6749,  1.6758,  1.6777,  ...,  1.4329,  1.4681,  1.5015],
          [ 1.6609,  1.6624,  1.6695,  ...,  1.4710,  1.5065,  1.5143],
          ...,
          [-0.2446, -0.2617, -0.2779,  ..., -0.2630, -0.2951, -0.2858],
          [-0.3710, -0.3652, -0.3236,  ..., -0.2629, -0.3425, -0.3705],
          [-0.4187, -0.3553, -0.3099,  ..., -0.2710, -0.3734, -0.4234]],
 
         [[ 1.9777,  1.9785,  2.0046,  ...,  1.7101,  1.7694,  1.8033],
          [ 1.9769,  1.9777,

In [60]:
torch.nn.Softmax(dim=1)(torch.tensor([[-10., 10.], [1., 1.]]))

tensor([[2.0612e-09, 1.0000e+00],
        [5.0000e-01, 5.0000e-01]])

In [8]:
import ignite

ModuleNotFoundError: No module named 'ignite'

In [25]:
from torchvision.models import resnet18, resnet50

oi = resnet18()

In [27]:
oi

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  